In [ ]:
# Let's see if we can leverage the ProcessPoolExecutor to launch a bunch of optimizations simultaneously.
#
import concurrent.futures

from mlos.Optimizers.BayesianOptimizerConfigStore import bayesian_optimizer_config_store
from mlos.Optimizers.BayesianOptimizerFactory import BayesianOptimizerFactory
from mlos.Optimizers.OptimizationProblem import OptimizationProblem, Objective
from mlos.Optimizers.OptimumDefinition import OptimumDefinition

from mlos.OptimizerEvaluationTools.ObjectiveFunctionFactory import ObjectiveFunctionFactory
from mlos.OptimizerEvaluationTools.ObjectiveFunctionConfigStore import objective_function_config_store


def run_optimization(optimizer_config, objective_function_config, num_iterations):
    objective_function = ObjectiveFunctionFactory.create_objective_function(objective_function_config)
    optimizer_factory = BayesianOptimizerFactory()
    optimizer = optimizer_factory.create_local_optimizer(
        optimizer_config=optimizer_config,
        optimization_problem=OptimizationProblem(
            parameter_space=objective_function.parameter_space,
            objective_space=objective_function.output_space,
            objectives=[Objective(name='y', minimize=True)]
        )
    )
    for i in range(num_iterations):
        parameters = optimizer.suggest()
        objectives = objective_function.evaluate_point(parameters)
        optimizer.register(parameters.to_dataframe(), objectives.to_dataframe())
    return optimizer.optimum(OptimumDefinition.BEST_OBSERVATION)
        

with concurrent.futures.ProcessPoolExecutor(max_workers=8) as executor:
    future = executor.submit(
        run_optimization,
        bayesian_optimizer_config_store.default,
        objective_function_config_store.get_config_by_name('three_level_quadratic'),
        num_iterations=10
    )
    